In [ ]:
import jax
import jax.numpy as jnp
import jax.random as jrandom
import numpy as np
import os

from penzai import pz
from tqdm.notebook import tqdm

from jaxl.constants import *
from jaxl.models.utils import get_model, load_config, load_params, get_wsrl_model
from jaxl.models.policies import get_policy, policy_output_dim
from jaxl.buffers import get_buffer
from jaxl.utils import get_device, parse_dict

import IPython

pz.ts.register_as_default()

# Optional automatic array visualization extras:
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()
pz.ts.active_autovisualizer.set_interactive(pz.ts.ArrayAutovisualizer())

get_device("gpu:0")

In [ ]:
result_dir = "/home/bryan/research/jaxl/logs/manipulator_learning"

ablation_name = "stack"
# learner_name = "bc-10k_steps-06-04-24_10_06_35-333b32a8-c019-4fed-9b8f-1ce59166bb2b"
learner_name = "warm_start_policy_evaluation-06-05-24_10_50_37-f9950c16-2b62-4078-85fd-2e43f5e6d1ed"

learner_path = os.path.join(result_dir, ablation_name, learner_name)

In [ ]:
_, config = load_config(learner_path)
config

In [ ]:
buffer_config = parse_dict(dict(
    load_buffer='/home/bryan/research/lfgp/lfgp_data/custom_expert_data/stack/1000000_steps_no_extra_final/int_0.gz',
    buffer_type='default',
    set_size=False,
))

In [ ]:
buffer = get_buffer(buffer_config)

In [ ]:
model_out_dim = policy_output_dim(buffer.act_dim, config.learner_config)

param_key = CONST_POLICY
if config.learner_config.learner == CONST_BC:
    model = get_model(
        int(np.prod(buffer.input_dim)),
        buffer.act_dim,
        config.model_config
    )
    include_absorbing_state = True
elif config.learner_config.task == CONST_WSRL:
    if config.learner_config.learner == CONST_POLICY_EVALUATION:
        model = get_model(
            int(np.prod(buffer.input_dim)),
            (1,),
            config.model_config.vf
        )
        include_absorbing_state = True
        param_key = CONST_VF
    else:
        model = get_wsrl_model(
            int(np.prod(buffer.input_dim)),
            model_out_dim,
            config.model_config.policy
        )
        include_absorbing_state = True
else:
    model = get_model(
        int(np.prod(buffer.input_dim)) - 1,
        model_out_dim,
        config.model_config.policy
    )
    include_absorbing_state = False

params = load_params(f"{learner_path}:latest")

## Check Dormant

In [ ]:
obss = buffer.sample(256)[0]

if not include_absorbing_state:
    obss = obss[..., :-1]

print(obss.shape)

In [ ]:
out, state = model.model.apply(params[CONST_MODEL_DICT][CONST_MODEL][CONST_VF], obss, capture_intermediates=True, mutable=["intermediates"], eval=True)

In [ ]:
res = dict()
for (kp, val) in jax.tree_util.tree_flatten_with_path(state["intermediates"])[0]:
    if getattr(kp[0], "key", False) == "__call__":
        continue
    per_neuron_score = jnp.mean(jnp.abs(val), axis=0)
    curr_key = "/".join([curr_kp.key if hasattr(curr_kp, "key") else str(curr_kp.idx) for curr_kp in kp][:-2])
    res[curr_key] = (per_neuron_score / jnp.mean(per_neuron_score, axis=-1))
    if np.prod(res[curr_key].shape) % 4 == 0:
        res[curr_key] = res[curr_key].reshape((4, -1))


In [ ]:
res

In [ ]:
np.mean(out)

In [ ]:

for (kp, val) in jax.tree_util.tree_flatten_with_path(params[CONST_MODEL_DICT][CONST_MODEL][CONST_POLICY])[0]:
    print(kp, val.shape)
    

## Visualize Parameters

In [ ]:
{
    "/".join([curr_kp.key for curr_kp in kp]): pz.nx.wrap(val) for (kp, val) in jax.tree_util.tree_flatten_with_path(params["model_dict"]["model"]["policy"])[0]
}